# Package install

In [ ]:
# install Hugging Face Libraries
! pip install -q datasets
! pip install -q transformers -U
! pip install -q transformers[torch] -U
! pip install -q sentencepiece
! pip install -q evaluate
! pip install -q bert_score
! pip install -q -U accelerate
! pip install -q -U bitsandbytes
! pip install -q peft
! pip install -q wandb

In [ ]:
from huggingface_hub import notebook_login

notebook_login()


# Process Dataset




In [ ]:
dataset_id = "minh21/cpgQA-v1.0-unique-context-test-10-percent-validation-10-percent"

In [ ]:
# Load your dataset
from datasets import load_dataset

dataset = load_dataset(dataset_id)
# dataset = dataset.remove_columns(["title", "id"])
dataset = dataset.rename_column("answer_text", "answer")

In [ ]:
train_dataset = dataset["train"]
test_dataset = dataset["test"]
validation_dataset = dataset["validation"]

In [ ]:
test_dataset[0]

In [ ]:
validation_dataset[0]

# Model Identification

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

model_id = "google/flan-t5-large"
checkpoint = "flant5-large-lora"
username = "espiusedwards"
# model_id="facebook/bart-large"
# Load tokenizer of FLAN-t5-large
tokenizer = AutoTokenizer.from_pretrained(model_id)
max_length = 512
max_target_length = 200

In [ ]:
import evaluate


def evaluate_qa(predicted_result):

    squad_metric = evaluate.load("squad")
    predictions = [
        {"prediction_text": answer, "id": str(id)}
        for id, answer in enumerate(predicted_result["output"])
    ]
    references = [
        {
            "answers": {"answer": [ds["answer"]], "text": [ds["answer"]]},
            "id": str(id),
        }
        for id, ds in enumerate(predicted_result)
    ]
    results = {}
    results["squad"] = squad_metric.compute(
        predictions=predictions, references=references
    )

    predictions = predicted_result["output"]
    references = predicted_result["answer"]

    bleu_metrics = evaluate.load("bleu")
    results["bleu"] = bleu_metrics.compute(
        predictions=predictions, references=references
    )

    bertscore_metric = evaluate.load("bertscore")
    berscore = bertscore_metric.compute(
        predictions=predictions, references=references, lang="en"
    )
    results["bertscore"] = {
        "precision": sum(berscore["precision"]) / len(berscore["precision"]),
        "recall": sum(berscore["recall"]) / len(berscore["recall"]),
        "f1": sum(berscore["f1"]) / len(berscore["f1"]),
    }

    return results

# Evluate Prompting

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch

tokenizer = AutoTokenizer.from_pretrained(model_id, device_map="auto")
model = AutoModelForSeq2SeqLM.from_pretrained(model_id, device_map="auto")

In [ ]:
max_token = 0
item_with_tokens_size_larger_than_limit = []
for data in train_dataset:
    text = data["context"] + "\n" + data["question"]
    l = tokenizer(text)["input_ids"].__len__()
    if l >= 512:
        item_with_tokens_size_larger_than_limit.append(data)
    if l >= max_token:
        max_token = l

print(max_token)

In [ ]:
def interfere(ds):
    ds["input"] = (
        f"context: {dataset['train'][0]['context']} "
        + f"question: {dataset['train'][0]['question']} "
        + f"answer: {dataset['train'][0]['answer']}\n"
        + f"context: {ds['context']} "
        + f"question: {ds['question']} "
        + f"answer: "
    )
    input_ids = tokenizer(
        ds["input"],
        return_tensors="pt",
        padding="max_length",
        truncation=True,
        max_length=max_length,
    ).input_ids.to(torch.device("cuda"))
    outputs = model.generate(input_ids, max_new_tokens=max_target_length)
    ds["output"] = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return ds


model.eval()
with torch.no_grad():
    predicted_result = dataset["test"].map(interfere, batched=False)

In [ ]:
evaluate_qa(predicted_result)

# Tokenize data

In [ ]:
# def tokenize(batch):
#     inputs = tokenizer(f"[CONTEXT]: {batch['context']} \n [QUESTION]: {batch['question']}" , padding= True, truncation = True, max_length = max_length, return_tensors='pt')
#     targets = tokenizer(batch['answer'], padding= True, truncation = True, max_length = max_length, return_tensors='pt')
#     inputs['labels'] = targets['input_ids']
#     return inputs
def tokenize(batch):
    input = f"question: {batch['question']} " f"context: {batch['context']}"
    inputs = tokenizer(input, truncation=True, max_length=max_length)
    targets = tokenizer(batch["answer"], truncation=True, max_length=max_length)
    inputs["labels"] = targets["input_ids"]
    return inputs

In [ ]:
tokenized_train_dataset = train_dataset.map(
    tokenize, batched=False, remove_columns=train_dataset.column_names
)
tokenized_test_dataset = test_dataset.map(
    tokenize, batched=False, remove_columns=test_dataset.column_names
)
tokenized_validation_dataset = validation_dataset.map(
    tokenize, batched=False, remove_columns=test_dataset.column_names
)

In [ ]:
tokenized_validation_dataset[0]

In [ ]:
first_context_decoded = tokenizer.decode(
    tokenized_train_dataset[0]["input_ids"], skip_special_tokens=True
)
print(first_context_decoded)

In [ ]:
train_dataset[0]

# # Compute Metric

In [ ]:
import evaluate

squad_metric = evaluate.load("squad")
bleu_metrics = evaluate.load("bleu")


def compute_metrics(pred):
    labels_ids, pred_ids = pred

    pred_ids[pred_ids == -100] = tokenizer.pad_token_id
    predictions = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)

    labels_ids[labels_ids == -100] = tokenizer.pad_token_id
    references = tokenizer.batch_decode(labels_ids, skip_special_tokens=True)

    squad_predictions = [
        {"prediction_text": answer, "id": str(id)}
        for id, answer in enumerate(predictions)
    ]
    squad_references = [
        {
            "answers": {"answer_start": [-1], "text": [answer]},
            "id": str(id),
        }
        for id, answer in enumerate(references)
    ]
    results = {}
    results["squad"] = squad_metric.compute(
        predictions=squad_predictions, references=squad_references
    )

    results["bleu"] = bleu_metrics.compute(
        predictions=predictions, references=references
    )

    return results

# Fine-tune and evaluate FLAN-T5

In [ ]:
from transformers import AutoModelForSeq2SeqLM

# model_id="google/flan-t5-large"
# checkpoint = "flant5-large-lora"
# username = "espiusedwards"

# load model from the hub
model = AutoModelForSeq2SeqLM.from_pretrained(model_id)
# model = AutoModelForSeq2SeqLM.from_pretrained(model_id, load_in_8bit=True, device_map="auto")

In [ ]:
from peft import LoraConfig, get_peft_model, prepare_model_for_int8_training, TaskType

# Define LoRA Config
lora_config = LoraConfig(
    r=24,
    lora_alpha=32,
    target_modules=["q", "v"],
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.SEQ_2_SEQ_LM,
)
# prepare int-8 model for training
# model = prepare_model_for_int8_training(model)

# add LoRA adaptor
model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

In [ ]:
from transformers import DataCollatorForSeq2Seq

# we want to ignore tokenizer pad token in the loss
label_pad_token_id = -100
# Data collator
data_collator = DataCollatorForSeq2Seq(
    tokenizer,
    model=model,
    label_pad_token_id=label_pad_token_id,
    pad_to_multiple_of=8,
    return_tensors="pt",
)

In [ ]:
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments

output_dir = f"./{checkpoint}"

# Define training args
training_args = Seq2SeqTrainingArguments(
    output_dir=output_dir,
    auto_find_batch_size=True,
    learning_rate=1e-3,  # higher learning rate
    num_train_epochs=5,
    logging_dir=f"{output_dir}/logs",
    logging_strategy="steps",
    evaluation_strategy="steps",
    eval_steps=100,
    logging_steps=100,
    save_strategy="no",
    predict_with_generate=True,
    # report_to="wandb",
    generation_max_length=max_target_length,
)

# Create Trainer instance
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=tokenized_train_dataset.with_format("torch"),
    eval_dataset=tokenized_validation_dataset.with_format("torch"),
    compute_metrics=compute_metrics,
)
model.config.use_cache = False  # silence the warnings. Please re-enable for inference!

In [ ]:
# train model
trainer.train()

In [ ]:
model.push_to_hub(
    "espiusedwards/flant5-large-lora",
    use_auth_token=True,
    commit_message="fix prompt, r = 24, 5 epoch",
    private=True,
    create_pr=1,
)

In [ ]:
trainer.evaluate()

In [ ]:
# # Save our LoRA model & tokenizer results
# peft_model_id="results"
# trainer.model.save_pretrained(peft_model_id)
# tokenizer.save_pretrained(peft_model_id)
# # if you want to save the base model to call
# trainer.model.base_model.save_pretrained(peft_model_id)

# Evaluate

In [ ]:
# Load adapters from hub
import torch
from peft import PeftModel, PeftConfig
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

# Load peft config for pre-trained checkpoint etc.
peft_model_id = "espiusedwards/flant5-large-lora"
config = PeftConfig.from_pretrained(peft_model_id)

# load base LLM model and tokenizer
model = AutoModelForSeq2SeqLM.from_pretrained(
    config.base_model_name_or_path, load_in_8bit=True, device_map={"": 0}
)
tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)

# Load the Lora model
model = PeftModel.from_pretrained(model, peft_model_id, device_map={"": 0})
model.eval()

print("Peft model loaded")

In [ ]:
# checkpoint_model = trainer.model

In [ ]:
"""
# Evaluate from checkpoint
from transformers import AutoTokenizer
from peft import AutoPeftModelForSeq2SeqLM

model_checkpoint = f"{username}/{checkpoint}"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, device_map="auto")
checkpoint_model = AutoPeftModelForSeq2SeqLM.from_pretrained(
    model_checkpoint, device_map="auto"
)
"""

In [ ]:
import torch


def interfere_prompt(ds):
    qa_input = f"question: {ds['question']} " f"context: {ds['context']}"
    input_ids = tokenizer(
        qa_input,
        return_tensors="pt",
        padding="max_length",
        truncation=True,
        max_length=max_length,
    ).input_ids.to(torch.device("cuda"))
    outputs = model.generate(input_ids=input_ids, max_new_tokens=max_target_length)
    ds["output"] = tokenizer.decode(outputs[0]).split(">")[1].split("<")[0].strip()
    return ds


model.eval()
with torch.no_grad():
    predicted_result = dataset["test"].map(interfere_prompt, batched=False)

In [ ]:
evaluate_qa(predicted_result)

# Other Evaluation

In [ ]:
model.to("cuda")
predictions_for_squad = []
predictions_for_squad_v2 = []
predictions_for_bert_score = []
references_for_bert_score = []
for data in test_dataset:
    context = data["context"]
    question = data["question"]
    answer = data["answer"]
    id = data["id"]
    input = f"""
    Read this and answer the question. If the question is unanswerable, "say \"unanswerable\".\n\n{question}\n\n{context}"
    """
    model_inputs = tokenizer(
        input,
        padding="max_length",
        max_length=512,
        truncation=True,
        return_tensors="pt",
    ).to(torch.device("cuda"))

    model_output = tokenizer.decode(
        model.generate(
            input_ids=model_inputs["input_ids"],
            attention_mask=model_inputs["attention_mask"],
        )[0],
        skip_special_tokens=True,
    )

    predictions_for_squad.append(
        {
            "prediction_text": model_output,
            "id": str(id),
        }
    )

    predictions_for_squad_v2.append(
        {
            "prediction_text": model_output,
            "no_answer_probability": 0,
            "id": str(id),
        }
    )

    predictions_for_bert_score.append(model_output)
    references_for_bert_score.append(answer)
    # predictions.extend(predicted_texts)

In [ ]:
references_for_squad_v2 = [
    {
        "answers": {"answer_start": [ds["answer_start"]], "text": [ds["answer"]]},
        "id": str(ds["id"]),
    }
    for id, ds in enumerate(test_dataset)
]

In [ ]:
from evaluate import load

results = dict()
squad_metric = load("squad_v2")
results["squad_v2"] = squad_metric.compute(
    predictions=predictions_for_squad_v2, references=references_for_squad_v2
)

squad_metric = load("squad")
results["squad"] = squad_metric.compute(
    predictions=predictions_for_squad, references=references_for_squad_v2
)

bleu_metrics = load("bleu")
results["bleu"] = bleu_metrics.compute(
    predictions=predictions_for_bert_score, references=references_for_bert_score
)

bertscore_metric = load("bertscore")
berscore = bertscore_metric.compute(
    predictions=predictions_for_bert_score,
    references=references_for_bert_score,
    lang="en",
)

results["bertscore"] = {
    "precision": sum(berscore["precision"]) / len(berscore["precision"]),
    "recall": sum(berscore["recall"]) / len(berscore["recall"]),
    "f1": sum(berscore["f1"]) / len(berscore["f1"]),
}
results

# Too small loss => Validation check on train dataset

In [ ]:
# validation set
validation_dataset = train_dataset.select(range(100))
validation_dataset

In [ ]:
model.to("cuda")
predictions_for_squad = []
predictions_for_squad_v2 = []
predictions_for_bert_score = []
references_for_bert_score = []
for data in validation_dataset:
    context = data["context"]
    question = data["question"]
    answer = data["answer"]
    id = data["id"]
    input = f"""
    Read this and answer the question. If the question is unanswerable, "say \"unanswerable\".\n\n{context}\n\n{question}"
    """
    model_inputs = tokenizer(
        input,
        padding="max_length",
        max_length=512,
        truncation=True,
        return_tensors="pt",
    ).to(torch.device("cuda"))

    model_output = tokenizer.decode(
        model.generate(
            input_ids=model_inputs["input_ids"],
            attention_mask=model_inputs["attention_mask"],
        )[0],
        skip_special_tokens=True,
    )

    predictions_for_squad.append(
        {
            "prediction_text": model_output,
            "id": str(id),
        }
    )

    predictions_for_squad_v2.append(
        {
            "prediction_text": model_output,
            "no_answer_probability": 0,
            "id": str(id),
        }
    )

    predictions_for_bert_score.append(model_output)
    references_for_bert_score.append(answer)
    # predictions.extend(predicted_texts)

In [ ]:
references_for_squad_v2 = [
    {
        "answers": {"answer_start": [ds["answer_start"]], "text": [ds["answer"]]},
        "id": str(ds["id"]),
    }
    for id, ds in enumerate(validation_dataset)
]

In [ ]:
from evaluate import load

results = dict()
squad_metric = load("squad_v2")
results["squad_v2"] = squad_metric.compute(
    predictions=predictions_for_squad_v2, references=references_for_squad_v2
)

squad_metric = load("squad")
results["squad"] = squad_metric.compute(
    predictions=predictions_for_squad, references=references_for_squad_v2
)

bleu_metrics = load("bleu")
results["bleu"] = bleu_metrics.compute(
    predictions=predictions_for_bert_score, references=references_for_bert_score
)

bertscore_metric = load("bertscore")
berscore = bertscore_metric.compute(
    predictions=predictions_for_bert_score,
    references=references_for_bert_score,
    lang="en",
)

results["bertscore"] = {
    "precision": sum(berscore["precision"]) / len(berscore["precision"]),
    "recall": sum(berscore["recall"]) / len(berscore["recall"]),
    "f1": sum(berscore["f1"]) / len(berscore["f1"]),
}
results

# Draft - Try functions

In [ ]:
# import evaluate
# from transformers import T5ForConditionalGeneration, T5Tokenizer
# # Load the model and tokenizer
# model = T5ForConditionalGeneration.from_pretrained('espiusedwards/flant5-large-lora')
# tokenizer = T5Tokenizer.from_pretrained('t5-small') # adjust model size if necessary
# model.eval()

# # Load the metric
# metric = evaluate.load("squad_v2")

# # Assuming `test_data` is your test data
# # and test_data is a list of dictionaries with 'question', 'context', and 'id' keys

# # Get predictions
# predictions = []
# for i in range(len(tokenized_test_dataset)):
#     item = tokenized_test_dataset[i]
#     inputs = {'input_ids': item['input_ids'], 'attention_mask': item['attention_mask']}
#     outputs = model.generate(**inputs, max_length=512, num_beams=4, length_penalty=2.0, early_stopping=True)
#     prediction = tokenizer.decode(outputs[0], skip_special_tokens=True)
#     predictions.append({'prediction_text': prediction, 'id': item['id']})

# references = [{'answers': {'answer_start': [0], 'text': [item['answer']]}, 'id': item['id']} for item in tokenized_test_dataset]

# # Compute the metric
# result = metric.compute(predictions=predictions, references=references)

# # Display the result
# print(result)

In [ ]:
# # evaluate
# import evaluate
# import numpy as np
# from datasets import load_from_disk
# from tqdm import tqdm

# # Metric
# metric = evaluate.load("squad_v2")

# def evaluate_peft_model(sample,max_target_length=50):
#     # generate summary
#     outputs = model.generate(input_ids=sample["input_ids"].unsqueeze(0).cuda(), do_sample=True, top_p=0.9, max_new_tokens=max_target_length)
#     prediction = tokenizer.decode(outputs[0].detach().cpu().numpy(), skip_special_tokens=True)
#     # decode eval sample
#     # Replace -100 in the labels as we can't decode them.
#     labels = np.where(sample['labels'] != -100, sample['labels'], tokenizer.pad_token_id)
#     labels = tokenizer.decode(labels, skip_special_tokens=True)

#     # Some simple post-processing
#     return prediction, labels

# # dataset: test_dataset

# # run predictions
# # this can take ~45 minutes
# predictions, references = [] , []
# for sample in tqdm(test_dataset):
#     p,l = evaluate_peft_model(sample)
#     predictions.append(p)
#     references.append(l)

# # compute metric
# squad_v2 = metric.compute(predictions=predictions, references=references)

# # print results
# print(f"Exact: {squad_v2['exact']}")
# print(f"f1: {squad_v2['f1']}")

In [ ]:
# from huggingface_hub import HfFolder
# from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments
# # Hugging Face repository id
# repository_id = f"{model_id.split('/')[1]}-{dataset_id}"
# # Define training args
# training_args = Seq2SeqTrainingArguments(
#     output_dir=repository_id,
#     per_device_train_batch_size=8,
#     per_device_eval_batch_size=8,
#     predict_with_generate=True,
#     fp16=False, # Overflows with fp16
#     learning_rate=5e-5,
#     num_train_epochs=5,
#     # logging & evaluation strategies
#     logging_dir=f"{repository_id}/logs",
#     logging_strategy="steps",
#     logging_steps=500,
#     evaluation_strategy="epoch",
#     save_strategy="epoch",
#     save_total_limit=2,
#     load_best_model_at_end=True,
#     # metric_for_best_model="overall_f1",
#     # push to hub parameters
#     report_to="tensorboard",
#     push_to_hub=False,
#     hub_strategy="every_save",
#     #hub_model_id=repository_id,
#     hub_token=HfFolder.get_token(),
# )

In [ ]:
# Compute metric
# import evaluate
# import numpy as np

# # Metric
# metric = evaluate.load("squad_v2")

# def compute_metrics(eval_preds):
#     preds, labels = eval_preds
#     if isinstance(preds, tuple):
#         preds = preds[0]

#     # Convert predictions and labels to the format expected by squad_v2 metric
#     predictions = [{'prediction_text': pred, 'id': i} for i, pred in enumerate(preds)]
#     references = [{'answers': {'answer_start': [0], 'text': [label]}, 'id': i} for i, label in enumerate(labels)]

#     result = metric.compute(predictions=predictions, references=references)
#     return result

In [ ]:
# Compute metric
# import evaluate
# import numpy as np
# import nltk
# from nltk.tokenize import sent_tokenize
# nltk.download("punkt")

# # Metric
# metric = evaluate.load("squad")

# def compute_metrics(eval_preds):
#     preds, labels = eval_preds
#     if isinstance(preds, tuple):
#         preds = preds[0]
#     decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
#     # Replace -100 in the labels as we can't decode them.
#     labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
#     decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

#     result = metric.compute(predictions=decoded_preds, references=decoded_labels)
#     result = {k: round(v * 100, 4) for k, v in result.items()}
#     prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
#     result["gen_len"] = np.mean(prediction_lens)
#     return result

In [ ]:
# # Create Trainer instance
# trainer = Seq2SeqTrainer(
#     model=model,
#     args=training_args,
#     data_collator=data_collator,
#     train_dataset=tokenized_train_dataset,
#     eval_dataset=tokenized_test_dataset,
#     compute_metrics=compute_metrics,
# )

In [ ]:
# Start training
# trainer.train()